In [1]:
!pip install nltk spacy
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 6.3 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.2/653.2 kB 15.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.1/741.1 kB 17.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [spacy]m14/15 [spacy]tion]b]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.7 MB/s  0:00:01m eta 0:00:01

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [44]:
import nltk 
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn. naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
import re
import string
from nltk.stem import PorterStemmer, WordNetLemmatizer

# ensure nltk resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords

# load spacy english model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/apple/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
train_path = "./Corona Virus/Corona_NLP_train.csv"
test_path = "./Corona Virus/Corona_NLP_test.csv"

train = pd.read_csv(train_path, encoding='latin1')
test = pd.read_csv(test_path, encoding='latin1')

In [6]:
train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [7]:
test.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [13]:
# top 5 tweets and sentiment in training data
top_5 = train['OriginalTweet'].tolist()[:5]
top_5_sentiments = train['Sentiment'].tolist()[:5]

for id, tweet in enumerate(top_5):
    print(f"Tweet {id+1}:")
    print(tweet)
    print(f"Sentiment: {top_5_sentiments[id]}")
    print("\n")

Tweet 1:
@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8
Sentiment: Neutral


Tweet 2:
advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order
Sentiment: Positive


Tweet 3:
Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P
Sentiment: Positive


Tweet 4:
My food stock is not the only one which is empty...

PLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. 
Stay calm, stay safe.

#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j
Sentiment: Positive


Tweet 5:
Me, ready to go at supermarket during the #COVID19 outbreak.

Not because I'm paranoid, b

In [14]:
# top 5 tweets and sentiment in test data
top_5 = test['OriginalTweet'].tolist()[:5]
top_5_sentiments = test['Sentiment'].tolist()[:5]

for id, tweet in enumerate(top_5):
    print(f"Tweet {id+1}:")
    print(tweet)
    print(f"Sentiment: {top_5_sentiments[id]}")
    print("\n")

Tweet 1:
TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as #coronavirus-fearing shoppers stock up https://t.co/Gr76pcrLWh https://t.co/ivMKMsqdT1
Sentiment: Extremely Negative


Tweet 2:
When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how  #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY
Sentiment: Positive


Tweet 3:
Find out how you can protect yourself and loved ones from #coronavirus. ?
Sentiment: Extremely Positive


Tweet 4:
#Panic buying hits #NewYork City as anxious shoppers stock up on food&amp;medical supplies after #healthcare worker in her 30s becomes #BigApple 1st confirmed #coronavirus patient OR a #Bloomberg staged event?

https://t.co/IASiReGPC4

#QAnon #QAnon2018 #QAnon2020 
#Election2020 #CDC https://t.co/29isZOewxu
Sentiment: Negative


Tweet 5:
#toiletpaper #dunnypaper #coronavirus #co

Timmy, timmy, timmY, TImmy

timmy

In [26]:
one = top_5[0]
one

'TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as #coronavirus-fearing shoppers stock up https://t.co/Gr76pcrLWh https://t.co/ivMKMsqdT1'

In [27]:
#converting all texts to lowercase
one = one.lower()
one

'trending: new yorkers encounter empty supermarket shelves (pictured, wegmans in brooklyn), sold-out online grocers (foodkick, maxdelivery) as #coronavirus-fearing shoppers stock up https://t.co/gr76pcrlwh https://t.co/ivmkmsqdt1'

In [28]:
# removing phone numbers 
sample = "Call me at 080-123-4567 regarding the pest outbreak."
cleaned_sample = re.sub(r'\d{3}[-\s]?\d{3}[-\s]?\d{4}', '', sample)
print(cleaned_sample)

Call me at  regarding the pest outbreak.


In [34]:
# removing links from texts
one = re.sub(r'http\S+|www\S+|https\S+', '', one, flags=re.MULTILINE)
one

'trending: new yorkers encounter empty supermarket shelves (pictured, wegmans in brooklyn), sold-out online grocers (foodkick, maxdelivery) as #coronavirus-fearing shoppers stock up  '

In [35]:
# removing punctuation
one_new = one.translate(str.maketrans('', '', string.punctuation))
print(one_new)

trending new yorkers encounter empty supermarket shelves pictured wegmans in brooklyn soldout online grocers foodkick maxdelivery as coronavirusfearing shoppers stock up  


In [36]:
#removing stopwords
stopwords_list = set(stopwords.words('english'))

def remove_stopwords(text):
    return [word for word in text.split() if word not in stopwords_list]

cleaned_one = remove_stopwords(one_new)
cleaned_one

['trending',
 'new',
 'yorkers',
 'encounter',
 'empty',
 'supermarket',
 'shelves',
 'pictured',
 'wegmans',
 'brooklyn',
 'soldout',
 'online',
 'grocers',
 'foodkick',
 'maxdelivery',
 'coronavirusfearing',
 'shoppers',
 'stock']

In [37]:
# text cleaning function
def clean_text(text):
    # Convert to lowercase
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'<.*?>', '', text) # Remove HTML
    text = re.sub(r'@\w+', '', text) # Remove mentions
    text = re.sub(r'#\w+', '', text) # Remove hashtags
    # remove number
    text = re.sub(r'\d{3}[-\s]?\d{3}[-\s]?\d{4}', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text) # Remove special characters and numbers
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces
    return text

In [38]:
#cleaning the training and test data
train['Cleaned_Tweet'] = train['OriginalTweet'].apply(clean_text)
test['Cleaned_Tweet'] = test['OriginalTweet'].apply(clean_text)

In [40]:
train[['OriginalTweet', 'Cleaned_Tweet']].head()

,OriginalTweet,Cleaned_Tweet
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,and and
1,advice Talk to your neighbours family to excha...,advice talk to your neighbours family to excha...
2,Coronavirus Australia: Woolworths to give elde...,coronavirus australia woolworths to give elder...
3,My food stock is not the only one which is emp...,my food stock is not the only one which is emp...
4,"Me, ready to go at supermarket during the #COV...",me ready to go at supermarket during the outbr...


In [43]:
new_one = train['Cleaned_Tweet'].tolist()[:5]
new_one

['and and',
 'advice talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist gp set up online shopping accounts if poss adequate supplies of regular meds but not over order',
 'coronavirus australia woolworths to give elderly disabled dedicated shopping hours amid covid outbreak',
 'my food stock is not the only one which is empty please dont panic there will be enough food for everyone if you do not take more than you need stay calm stay safe',
 'me ready to go at supermarket during the outbreak not because im paranoid but because my food stock is litteraly empty the is a serious thing but please dont panic it causes shortage']

In [45]:
text=new_one[1]
text

'advice talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist gp set up online shopping accounts if poss adequate supplies of regular meds but not over order'

In [46]:
#tokenizing text
tokens = nltk.word_tokenize(text)
tokens

['advice',
 'talk',
 'to',
 'your',
 'neighbours',
 'family',
 'to',
 'exchange',
 'phone',
 'numbers',
 'create',
 'contact',
 'list',
 'with',
 'phone',
 'numbers',
 'of',
 'neighbours',
 'schools',
 'employer',
 'chemist',
 'gp',
 'set',
 'up',
 'online',
 'shopping',
 'accounts',
 'if',
 'poss',
 'adequate',
 'supplies',
 'of',
 'regular',
 'meds',
 'but',
 'not',
 'over',
 'order']

In [47]:
# removing stop words
cleaned_text = [word for word in tokens if word not in stopwords_list]
cleaned_text

['advice',
 'talk',
 'neighbours',
 'family',
 'exchange',
 'phone',
 'numbers',
 'create',
 'contact',
 'list',
 'phone',
 'numbers',
 'neighbours',
 'schools',
 'employer',
 'chemist',
 'gp',
 'set',
 'online',
 'shopping',
 'accounts',
 'poss',
 'adequate',
 'supplies',
 'regular',
 'meds',
 'order']

In [53]:
PorterStemmer().stem("slept")

'slept'

In [54]:
# stemming
ps = PorterStemmer()
stemmed_text = [ps.stem(word) for word in cleaned_text]
stemmed_text

['advic',
 'talk',
 'neighbour',
 'famili',
 'exchang',
 'phone',
 'number',
 'creat',
 'contact',
 'list',
 'phone',
 'number',
 'neighbour',
 'school',
 'employ',
 'chemist',
 'gp',
 'set',
 'onlin',
 'shop',
 'account',
 'poss',
 'adequ',
 'suppli',
 'regular',
 'med',
 'order']

In [66]:
WordNetLemmatizer().lemmatize("smokes")

'smoke'

In [59]:
# lemmatization
lemmatizer = WordNetLemmatizer()
lemmatized_text = [lemmatizer.lemmatize(word) for word in stemmed_text]
lemmatized_text

['advic',
 'talk',
 'neighbour',
 'famili',
 'exchang',
 'phone',
 'number',
 'creat',
 'contact',
 'list',
 'phone',
 'number',
 'neighbour',
 'school',
 'employ',
 'chemist',
 'gp',
 'set',
 'onlin',
 'shop',
 'account',
 'poss',
 'adequ',
 'suppli',
 'regular',
 'med',
 'order']

In [100]:
#using spacy

# Text with "water" used as a Noun and a Verb
text = "Timothy teaches Musonda and Thato; Data Science weekly"
doc = nlp(text)

In [101]:
doc

Timothy teaches Musonda and Thato; Data Science weekly

In [102]:
for tok in doc:
    print(tok.text, tok.pos_ , tok.dep_, spacy.explain(tok.pos_))

Timothy PROPN nsubj proper noun
teaches VERB ccomp verb
Musonda PROPN dobj proper noun
and CCONJ cc coordinating conjunction
Thato PROPN conj proper noun
; PUNCT punct punctuation
Data PROPN compound proper noun
Science PROPN nsubj proper noun
weekly NOUN ROOT noun


In [103]:
doc.ents

(Musonda, Thato, Data Science, weekly)

In [104]:
for tok in doc.ents:
    print(tok.text, tok.label_, spacy.explain(tok.label_))

Musonda PERSON People, including fictional
Thato ORG Companies, agencies, institutions, etc.
Data Science ORG Companies, agencies, institutions, etc.
weekly DATE Absolute or relative dates or periods


In [ ]:
data = []
for token in doc:
    if token.text.lower() == "water":
        description = spacy.explain(token.pos_)
        data.append([token.text, token.pos_, description])

df = pd.DataFrame(data, columns=["Word", "POS Tag", "Explanation"])
print("--- 1. POS TAGGING (Context Matters) ---")
print(df)

--- 1. POS TAGGING (Context Matters) ---
    Word POS Tag Explanation
0  water    NOUN        noun
1  water    VERB        verb


In [80]:
# Input Text: Contains Companies, Locations, Money, and Verbs
text = "Dangote Fertilizer is selling Urea for N15,000 in Lagos."
doc = nlp(text)

# --- PART A: POS TAGGING (Grammar) ---
# Goal: Find the Verbs and Nouns
pos_data = []
for token in doc:
    # We get the simple tag (pos_) and the detailed tag (tag_)
    pos_data.append([
        token.text,
        token.pos_,
        spacy.explain(token.pos_)
    ])

print("\n--- A. POS TAGGING (Grammar & Syntax) ---")
df_pos = pd.DataFrame(pos_data, columns=["Word", "POS Tag", "Meaning"])
print(df_pos)


--- A. POS TAGGING (Grammar & Syntax) ---
         Word POS Tag      Meaning
0     Dangote   PROPN  proper noun
1  Fertilizer   PROPN  proper noun
2          is     AUX    auxiliary
3     selling    VERB         verb
4        Urea   PROPN  proper noun
5         for     ADP   adposition
6     N15,000   PROPN  proper noun
7          in     ADP   adposition
8       Lagos   PROPN  proper noun
9           .   PUNCT  punctuation


In [81]:
# --- PART B: NER (Named Entities) ---
# Goal: Find the Real-World Objects (Who, Where, How Much)
ner_data = []
for ent in doc.ents:
    ner_data.append([
        ent.text,
        ent.label_,
        spacy.explain(ent.label_)
    ])

print("\n--- B. NAMED ENTITY RECOGNITION (Real World Objects) ---")
df_ner = pd.DataFrame(ner_data, columns=["Entity Text", "Label", "Description"])
print(df_ner)


--- B. NAMED ENTITY RECOGNITION (Real World Objects) ---
  Entity Text   Label                  Description
0  Fertilizer  PERSON  People, including fictional
1        Urea     GPE    Countries, cities, states
2     N15,000     GPE    Countries, cities, states
3       Lagos     GPE    Countries, cities, states
